In [23]:
import dlib
import cv2
import numpy as np
from imutils import face_utils
from EAR import eye_aspect_ratio
from MAR import mouth_aspect_ratio





In [ ]:

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./dlib_shape_predictor/shape_predictor_68_face_landmarks.dat')

EYE_AR_THRESH = 0.45
MOUTH_AR_THRESH = 0.79
HEADPOSE_THRESH = 25   

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(mStart, mEnd) = (49, 68)


In [ ]:

def predict_drowsiness(frame):
    """
    输入: BGR 图像 (cv2.imread读取的)
    输出: "Drowsy" 或 "Non Drowsy"
    """

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    if len(rects) == 0:
        return "Non Drowsy"   

    rect = rects[0]
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)

    # EAR
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    ear = (eye_aspect_ratio(leftEye) + eye_aspect_ratio(rightEye)) / 2.0

    # MAR
    mouth = shape[mStart:mEnd]
    mar = mouth_aspect_ratio(mouth)

    # Head Pose
    size = gray.shape
    # head_tilt_degree, _, _, _ = getHeadTiltAndCoords(size, np.zeros((6,2)), size[0])

    # Decision rules
    if ear < EYE_AR_THRESH:
        return "Drowsy"
    if mar > MOUTH_AR_THRESH:
        return "Drowsy"
    # if head_tilt_degree[0] > HEADPOSE_THRESH:
        # return "Drowsy"

    return "Non Drowsy"

In [ ]:
import os
import cv2
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from tqdm import tqdm

test_dir = "../data/splitted_Data/test"
classes = ["Non Drowsy", "Drowsy"]   

y_true = []
y_pred = []

for cls in classes:
    folder = os.path.join(test_dir, cls)
    label = cls  

    for img_name in tqdm(os.listdir(folder), desc=f"Processing {cls}"):
        if not img_name.endswith(".png"):
            continue

        img_path = os.path.join(folder, img_name)
        frame = cv2.imread(img_path)

        pred = predict_drowsiness(frame)

        y_true.append(label)
        y_pred.append(pred)

 
label_map = {"Non Drowsy": 0, "Drowsy": 1}
y_true = [label_map[l] for l in y_true]
y_pred = [label_map[p] for p in y_pred]

 
print("\n✔ Accuracy:", accuracy_score(y_true, y_pred))
print("✔ F1 Score:", f1_score(y_true, y_pred, average="binary"))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Non Drowsy", "Drowsy"]))

cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:\n", cm)


Processing Drowsy: 100%|██████████| 1118/1118 [00:05<00:00, 205.42it/s]


✔ Accuracy: 0.49115255858440937
✔ F1 Score: 0.11333333333333333

Classification Report:
              precision    recall  f1-score   support

  Non Drowsy       0.48      0.99      0.64       973
      Drowsy       0.83      0.06      0.11      1118

    accuracy                           0.49      2091
   macro avg       0.65      0.52      0.38      2091
weighted avg       0.67      0.49      0.36      2091


Confusion Matrix:
 [[ 959   14]
 [1050   68]]


Processing Drowsy: 100%|██████████| 1118/1118 [00:05<00:00, 195.55it/s]


KeyError: 'None Drowsy'